In [1]:
%matplotlib inline

from pathlib import Path
from time import process_time
import numpy as np
import pandas as pd
from mdptoolbox import mdp
from itertools import product
import gym

## Set up Gridworld

### States, Actions and Rewards

In [2]:
grid_size = (3, 4)
blocked_cell = (1, 1)
baseline_reward = -0.02
absorbing_cells = {(0, 3): 1, (1, 3): -1}

In [3]:
actions = ['L', 'U', 'R', 'D']
num_actions = len(actions)
probs = [.1, .8, .1, 0]

In [4]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)
to_2d = lambda x: np.unravel_index(x, grid_size)

In [5]:
num_states = np.product(grid_size)
cells = list(np.ndindex(grid_size))
states = list(range(len(cells)))

In [6]:
cell_state = dict(zip(cells, states))
state_cell= dict(zip(states, cells))

In [7]:
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}
blocked_state = to_1d(blocked_cell)

In [8]:
state_rewards = np.full(num_states, baseline_reward)
state_rewards[blocked_state] = 0
for state, reward in absorbing_states.items():
    state_rewards[state] = reward

In [9]:
action_outcomes = {}
for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

In [10]:
action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

### Transition Matrix

In [11]:
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

In [12]:
state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

In [13]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

In [14]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))
transitions = np.zeros((num_actions, num_states, num_states))
actions_ = list(range(num_actions))
for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

In [16]:
rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

## PyMDPToolbox

### Value Iteration

In [19]:
gamma = .99
epsilon = 1e-5

In [45]:
vi = mdp.ValueIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        epsilon=epsilon)

vi.run()
f'# Iterations: {vi.iter:,d} | Time: {vi.time:.4f}'

'# Iterations: 31 | Time: 0.0007'

In [46]:
policy = np.asarray([actions[i] for i in vi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [47]:
value = np.asarray(vi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,0.000000
1,0.848181,0.000000,0.714643,0.000000
2,0.808345,0.773328,0.736099,0.516083


### Policy Iteration

In [48]:
pi = mdp.PolicyIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        max_iter=1000)

pi.run()
f'# Iterations: {pi.iter:,d} | Time: {pi.time:.4f}'

'# Iterations: 7 | Time: 0.0021'

In [49]:
policy = np.asarray([actions[i] for i in pi.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [50]:
value = np.asarray(pi.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,1.594721e-16
1,0.848181,0.000000,0.714643,-0.000000e+00
2,0.808345,0.773328,0.736099,5.160828e-01


### Q Learning

In [30]:
start = process_time()
ql = mdp.QLearning(transitions=transitions,
                   reward=rewards,
                   discount=gamma,
                   n_iter=int(1e6))

ql.run()
f'Time: {process_time()-start:.4f}'

'Time: 11.9887'

In [31]:
policy = np.asarray([actions[i] for i in ql.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,R,U,L


In [32]:
value = np.asarray(ql.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.817386,0.922847,0.959248,0.000000
1,0.642177,0.000000,0.720226,0.000000
2,0.267461,0.419147,0.600977,0.258829


## Value Iteration

In [20]:
skip_states = list(absorbing_states.keys())+[blocked_state]
states_to_update = [s for s in states if s not in skip_states]

In [21]:
V = np.random.rand(num_states)
V[skip_states] = 0

iterations = 0
start = process_time()
while True:
    V_ = np.copy(V)
    for state in states_to_update:
        q_sa = np.sum(transitions[:, state] * (rewards[:, state] + gamma* V), axis=1)
        V[state] = np.max(q_sa)
    if np.sum(np.fabs(V - V_)) < epsilon:
        break
    iterations += 1
    if iterations % 1000 == 0:
        print(np.sum(np.fabs(V - V_)))

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 16 | Time 0.0117'

### Value Function

In [33]:
print(pd.DataFrame(V.reshape(grid_size)))

          0         1         2         3
0  0.884143  0.925054  0.961986  0.000000
1  0.848181  0.000000  0.714643  0.000000
2  0.808344  0.773327  0.736099  0.516082


In [34]:
np.allclose(V.reshape(grid_size), np.asarray(vi.V).reshape(grid_size))

True

### Optimal Policy

In [35]:
for state, reward in absorbing_states.items():
    V[state] = reward

policy = np.argmax(np.sum(transitions * V, 2),0)
policy

array([2, 2, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [36]:
pd.DataFrame(policy.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,L


## Policy Iteration

In [41]:
def policy_improvement(value, transitions):
    for state, reward in absorbing_states.items():
        value[state] = reward
    return np.argmax(np.sum(transitions * value, 2),0)

In [42]:
V = np.random.rand(num_states)
V[skip_states] = 0
pi = np.random.choice(list(range(num_actions)), size=num_states)

iterations = 0
start = process_time()
while True:
    pi_ = np.copy(pi)
    for state in states_to_update:
        action = policy[state]
        V[state] = np.dot(transitions[action, state], (rewards[action, state] + gamma* V))
        pi = policy_improvement(V.copy(), transitions)
    if np.array_equal(pi_, pi):
        break
    iterations += 1

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 3 | Time 0.0059'

In [43]:
pd.DataFrame(pi.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [44]:
pd.DataFrame(V.reshape(grid_size))

,0,1,2,3
0,0.765729,0.874981,0.923802,0.000000
1,0.697293,0.000000,0.405803,0.000000
2,0.625634,0.563798,0.506690,0.305024


## Q-Learning

In [517]:
max_episodes = 2500
alpha = .1
epsilon = .05

In [518]:
Q = np.random.rand(num_states, num_actions)
Q[skip_states] = 0

for episode in range(max_episodes):
    state = np.random.choice([s for s in states if s not in skip_states])
    while not state in absorbing_states.keys():
        if np.random.rand() < epsilon:
            action = np.random.choice(num_actions)
        else:
            action = np.argmax(Q[state])
        next_state = np.random.choice(states, p=transitions[action, state])
        reward = rewards[action, state, next_state]
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state])-Q[state, action])
        state = next_state

In [519]:
pd.DataFrame(np.argmax(Q, 1).reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,D


In [520]:
pd.DataFrame(np.max(Q, 1).reshape(grid_size))

,0,1,2,3
0,0.899691,0.934268,0.963701,0.000000
1,0.852410,0.000000,0.735339,0.000000
2,0.815526,0.778779,0.744708,0.559189
